# Zamiana jabłek na pomarańcze

CycleGAN - umożliwia przekształcenia obrazów w obu kierunkach, 4 modele, 2 dyskryminatory i 2 generatory; 

Generatory:
* G_AB - obrazy z domeny A do domeny B
* G_BA - obrazy z domeny B do domeny A

Dyskryminatory:
* d_A - różnica między autentycznymi obrazami A a wytworzonymi przez G_AB
* d_B - różnica między autentycznymi obrazami B a wytworzonymi przez G_BA


In [1]:
import os
import matplotlib.pyplot as plt

from models.cycleGAN import CycleGAN
from utils.loaders import DataLoader

W models.cycleGAN tworzymy obiekt CycleGAN który możemy nastepnie modyfikować
Generatory: U-Net (models.cycleGAN) , ResNet

U-Net
* downsampling - kompresowanie przestrzenne, ale rozbudowane pod względem kanałów
* upsampling - rozwijane przestrzennie, redukcja kanałów 
* Na wierzchołku u kontekstowe rozumienie czym jest obraz
* Pomijanie połączeń przepływ do dalszych warstw, scalenie stylu z zawartością obrazu
* Warstwa Concatenate - pomijanie połączeń, łączy downsampling z upsampling, liczba kanałów do 2k
* Warswa InstanceNormalization - normalizuje obserwacje indywidualnie (partia, warstwa, próbka, grupa), nie uczy wag, normalizacja obserwacji nie warstw

Dwie połówki:
-downsampling - Conv2D, krok 2
-upsampling - warstwy Concatenate


Dyskryminatory

Wyjście w postaci 8x8, podział na łaty i odgaduje czy są prawdziwe, prawdopodobieństwo dla każdej łaty, łaty oceniane równocześnie.

Zaleta: funkcja straty może mierzyć na podstawie stylu

* Zbiór warstw konwolucyjnych
* Wszystkie warstwy (oprócz 1) normalizacja 
* Ostatnia warstwa bez konwolucyjna z 1 filtrem, bez aktywacji

In [2]:
# run params
SECTION = 'paint'
RUN_ID = '0001'
DATA_NAME = 'apple2orange'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' # 'build' # 

**Dane**

In [3]:
IMAGE_SIZE = 128

In [4]:
data_loader = DataLoader(dataset_name=DATA_NAME, img_res=(IMAGE_SIZE, IMAGE_SIZE))

**Budowa sieci**

In [5]:
gan = CycleGAN(
    input_dim = (IMAGE_SIZE,IMAGE_SIZE,3)
    ,learning_rate = 0.0002
    , buffer_max_length = 50
    , lambda_validation = 1
    , lambda_reconstr = 10
    , lambda_id = 2
    , generator_type = 'unet'
    , gen_n_filters = 32
    , disc_n_filters = 32
    )

if mode == 'build':
    gan.save(RUN_FOLDER)
else:
    gan.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

g_AB - konwertowanie obrazu A na B

g_BA - konwetowanie obrazu B na A

d_A - różnica między autentycznymi obrazami A a fałszywymi generowanymi przez g_BA

d_B - różnica między autentycznymi obrazami A a fałszywymi generowanymi przez g_BA

In [6]:
gan.g_AB.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 64, 64, 32)   1568        input_3[0][0]                    
__________________________________________________________________________________________________
instance_normalization_6 (Insta (None, 64, 64, 32)   0           conv2d_10[0][0]                  
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 32)   0           instance_normalization_6[0][0]   
____________________________________________________________________________________________

In [7]:
gan.g_BA.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 64, 64, 32)   1568        input_4[0][0]                    
__________________________________________________________________________________________________
instance_normalization_13 (Inst (None, 64, 64, 32)   0           conv2d_18[0][0]                  
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 64, 64, 32)   0           instance_normalization_13[0][0]  
____________________________________________________________________________________________

In [8]:
gan.d_A.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 32)        1568      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        32832     
_________________________________________________________________
instance_normalization (Inst (None, 32, 32, 64)        0         
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       131200

In [9]:
gan.d_B.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 64, 32)        1568      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 64)        32832     
_________________________________________________________________
instance_normalization_3 (In (None, 32, 32, 64)        0         
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 128)       1312

Kompilacja dyskryminatory - bezpośrednio, mamy wejścia oraz binarne wyjścia

Kompilacja generatory - nie mamy sparowanych obrazów w zestawie danych, kryteria oceny:
* poprawność - czy można oszukać dyskryminatory

        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)
* rekonstrukcja - powrót do orginalnego obrazu
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
* tożsamość - obraz niezmieniony jeśli zastosujemy każdy z generatorów
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

In [10]:
BATCH_SIZE = 1
EPOCHS = 2
PRINT_EVERY_N_BATCHES = 10

TEST_A_FILE = 'n07740461_14740.jpg'
TEST_B_FILE = 'n07749192_4241.jpg'

* Szkolenie naprzemienne dyskryminatorów i generatorów
* Obrazy autentyczne 1, wygenerowane 0
        valid = np.ones
        fake = np.zeros
* Generator - fałszywe obrazy - trenowanie dyskryminatora
* Generatory szkolone razem

In [15]:
gan.train(data_loader
        , run_folder = RUN_FOLDER
        , epochs=EPOCHS
        , test_A_file = TEST_A_FILE
        , test_B_file = TEST_B_FILE
        , batch_size=BATCH_SIZE
        , sample_interval=PRINT_EVERY_N_BATCHES)

[Epoch 0/2] [Batch 0/995] [D loss: 1.280687, acc:  58%] [G loss: 16.858515, adv: 2.332528, recon: 1.192468, id: 1.300652] time: 0:00:00.185331 


TypeError: cannot pickle 'weakref' object

**Strata**

In [ ]:
fig = plt.figure(figsize=(20,10))

plt.plot([x[1] for x in gan.g_losses], color='green', linewidth=0.1) #DISCRIM LOSS
# plt.plot([x[2] for x in gan.g_losses], color='orange', linewidth=0.1)
plt.plot([x[3] for x in gan.g_losses], color='blue', linewidth=0.1) #CYCLE LOSS
# plt.plot([x[4] for x in gan.g_losses], color='orange', linewidth=0.25)
plt.plot([x[5] for x in gan.g_losses], color='red', linewidth=0.25) #ID LOSS
# plt.plot([x[6] for x in gan.g_losses], color='orange', linewidth=0.25)

plt.plot([x[0] for x in gan.g_losses], color='black', linewidth=0.25)

# plt.plot([x[0] for x in gan.d_losses], color='black', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.ylim(0, 5)

plt.show()